# AI-Assisted Multi-Reference Timekeeping Overview

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-org/ai-multi-reference-timekeeping/blob/main/notebooks/00_overview.ipynb)

This notebook validates that the time server scaffold can fuse sensor data with reference measurements.

In [ ]:
# Clone and install the repo when running in Colab
import os
if not os.path.exists('ai-multi-reference-timekeeping'):
    !git clone https://github.com/your-org/ai-multi-reference-timekeeping.git
%cd ai-multi-reference-timekeeping
!pip install -e .


In [ ]:
from ai_multi_reference_timekeeping.fusion import HeuristicFusion, Measurement, VirtualClock
from ai_multi_reference_timekeeping.kalman import ClockCovariance, ClockKalmanFilter, ClockState
from ai_multi_reference_timekeeping.time_server import (
    LightweightInferenceModel,
    SensorAggregator,
    TimeServer,
)

# Simple sensor that provides temperature readings
class DummySensor:
    def sample(self):
        return {'temperature_c': 22.0, 'humidity_pct': 45.0}

# Simple static reference clock
class StaticReference:
    def sample(self, frame):
        return Measurement(name='ref', offset=0.15, variance=0.2)

# Create Kalman filter for clock state estimation
kalman = ClockKalmanFilter(
    state=ClockState(offset=0.0, drift=0.0),
    covariance=ClockCovariance(p00=1.0, p01=0.0, p10=0.0, p11=1.0),
    process_noise_offset=1e-4,
    process_noise_drift=1e-6,
)

# Create virtual clock with heuristic fusion
clock = VirtualClock(kalman_filter=kalman, fusion=HeuristicFusion())

# Create time server with sensor, reference, and inference model
server = TimeServer(
    clock=clock,
    references=[StaticReference()],
    sensors=SensorAggregator(DummySensor()),
    inference=LightweightInferenceModel(),
)

# Execute one time step
update, frame, drift, drift_hint = server.step(1.0)
print(f'Fused offset: {update.fused_offset:.6f}')
print(f'Drift: {drift.drift:.9f}')
print(f'Drift hint: {drift_hint:.9f}')
print(f'Temperature: {frame.temperature_c}°C')
